In [ ]:
import rdflib
from flask import abort
from flask import make_response
from flask import request
from flask import Flask, jsonify
from flask_cors import CORS

In [ ]:
from flask import Flask, jsonify
from rdflib import Graph, Namespace, URIRef
from rdflib.plugins.sparql import prepareQuery

app = Flask(__name__)

# Load the Nobel ontology
g = Graph()
g.parse("data/nobel.owl")
g.parse("data/nobeldata.owl")



In [ ]:
from rdflib import Graph, Namespace

# Define the namespaces used in the ontology
nobel = Namespace('http://swat.cse.lehigh.edu/resources/onto/nobel.owl#')
rdf = Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#')
rdfs = Namespace('http://www.w3.org/2000/01/rdf-schema#')
owl = Namespace('http://www.w3.org/2002/07/owl#')
xml = Namespace('http://www.w3.org/XML/1998/namespace')

# Load the Nobel ontology from the OWL file
g = Graph()
g.parse('data/nobel.owl', format='xml')
g.parse("data/nobeldata.owl", format='xml')


# Query the ontology to print the information contained in it
for s, p, o in g:
    if isinstance(s, rdflib.term.URIRef) and isinstance(p, rdflib.term.URIRef) and isinstance(o, rdflib.term.URIRef):
        print(f"{s.n3()} {p.n3()} {o.n3()}")
    elif isinstance(s, rdflib.term.URIRef) and isinstance(p, rdflib.term.URIRef) and isinstance(o, rdflib.term.Literal):
        print(f"{s.n3()} {p.n3()} '{o.value}'^^{o.datatype.n3()}")


In [ ]:
from rdflib import Graph, Namespace, RDF, RDFS

# Load the ontology file
g = Graph()
g.parse('data/nobeldata.owl')

# Print the triples
for subj, pred, obj in g:
    print(subj, pred, obj)


In [ ]:

from rdflib import Graph, Namespace, RDF, RDFS

# create a graph and parse the nobel.owl ontology
g = Graph()
g.parse("data/nobeldata.owl")

# define the relevant namespaces
nobel = Namespace("http://swat.cse.lehigh.edu/resources/onto/nobel.owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")

# create a set to hold the names of all nations
nations = set()

# iterate over all instances of the PersonWinner class and add their nationality to the set
for person in g.subjects(RDF.type, nobel.PersonWinner):
    nation = g.value(person, nobel.nationality, None)
    if nation is not None:
        nations.add(nation)

# sort the set and print the names of all nations
for nation in sorted(nations):
    print(nation)


In [ ]:
from flask import Flask, jsonify
from rdflib import Graph, Namespace, RDF, RDFS

app = Flask(__name__)

# create a graph and parse the nobel.owl ontology
g = Graph()
g.parse("data/nobel.owl")
g.parse("data/nobeldata.owl")

# define the relevant namespaces
nobel = Namespace("http://swat.cse.lehigh.edu/resources/onto/nobel.owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")

# create a set to hold the names of all nations
nations = set()

# SPARQL query to retrieve the nationality of all PersonWinner instances
query = """
    PREFIX nobel: <http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>
    SELECT ?nation
    WHERE {
        ?person rdf:type nobel:PersonWinner .
        ?person nobel:nationality ?nation .
    }
"""

# execute the query and add the results to the nations set
for row in g.query(query):
    nations.add(row['nation'])
    
# sort the set and return the names of all nations as a JSON response
res = []
for row in g.query(query):
    res.append(row[0][0:])

res=list(set(res))
res.sort()
print({'ele':res})


In [ ]:
a

In [ ]:
@app.route('/nations',methods=['GET'])
def get_nations():
    return jsonify({'ele':"hi"})

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
#1st Query
'''PREFIX nobel: <http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT ?nation
    WHERE {
        ?person rdf:type nobel:PersonWinner .
        ?person nobel:nationality ?nation .
    }'''


#2nd Query
'''PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX base: <http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>

SELECT DISTINCT ?category_name
WHERE {
  ?category a owl:Class .
  FILTER(STRSTARTS(STR(?category), STR(base:))) .
  FILTER(!STRSTARTS(STR(?category), STR(base:NobelWinner))) .
  FILTER(!STRSTARTS(STR(?category), STR(base:Organization))) .
  FILTER(!STRSTARTS(STR(?category), STR(base:Prize))) .
  FILTER(!STRSTARTS(STR(?category), STR(base:OrganizationWinner))) .
  FILTER(!STRSTARTS(STR(?category), STR(base:PersonWinner))) .
  ?category rdfs:subClassOf base:Prize .
  BIND(REPLACE(STR(?category), STR(base:), "") AS ?category_name)
} 
ORDER BY ?category_name
'''

#3rd Query
'''PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX nobel: <http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>

SELECT DISTINCT ?year
WHERE {
  ?prize nobel:yearWon ?year .
} 
ORDER BY ?year
'''

In [1]:
from flask import Flask, render_template, request
import rdflib

g = rdflib.Graph()
g.parse("data/nobeldata.owl")
print("graph has %s statements." % len(g))

app = Flask(__name__)

@app.route("/")
def home():
    nationality = nations()
    category = categories()
    years = year()
    return render_template("index.html", nationality=nationality, categories = category, years = years)

@app.route("/nobel/categories")
def categories():
#GET: Return sorted names of all nobel categories
    qres = g.query(
            """
            PREFIX table:<http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>
            PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
            SELECT ?w
            {
                    ?g rdf:type table:PersonWinner;
                    table:WonPrize ?w.
                    }
            GROUP BY ?w
            ORDER BY ?w""")

    unique_category = []
    for row in qres:
        category = ("%s" % row).split('/')[-4]
        if category not in unique_category:
            unique_category.append(category)

    return unique_category

@app.route("/nobel/years")
def year():
#GET: Return sorted years nobels are awarded
    qres = g.query(
            """
            PREFIX table:<http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>
            PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
            SELECT ?w
            {
                    ?g rdf:type table:PersonWinner;
                    table:WonPrize ?w.
                    }
            GROUP BY ?w
            ORDER BY ?w""")

    unique_year = []

    for row in qres:
        year = ("%s" % row).split('/')[-2]
        if year not in unique_year:
            unique_year.append(year)

    unique_year.sort()

    return unique_year

@app.route("/nobel/nations")
def nations():
#GET: Return sorted names of all nations
    qres = g.query(
            """
            PREFIX table:<http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>
            PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
            SELECT ?n
            {
                    ?g rdf:type table:PersonWinner;
                    table:nationality ?n.
                    }
            GROUP BY ?n
            ORDER BY ?n""")

    nation = []
    for row in qres:
        name = ("%s" % row).rsplit('/',1)[-1]
        nation.append(name)

    return nation


@app.route("/nobel/getYear", methods=['GET', 'POST'])
def getYear():
#GET: Return list of all nobel winners for the given year
    year = request.form.get('year')

    qres = g.query(
        """
        PREFIX table:<http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>
        PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
        SELECT ?n
        {
        ?g rdf:type table:PersonWinner;
        table:name ?n;
        table:WonPrize ?w.
        ?w table:yearWon ?y.
        FILTER (?y = """ + str(year) + """)
        }""")

    winners = []
    for row in qres:
        winners.append("%s" % row)

    return render_template("winners.html", winners=winners)

@app.route("/nobel/getNation", methods=['GET', 'POST'])
def getNation():
#GET: Return list of all nobel winners for the given nation
    nation = request.form.get('nationality')

    qres = g.query(
        """
        PREFIX table:<http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>
        PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
        SELECT ?n ?d
        {
            ?g rdf:type table:PersonWinner;
            table:name ?n;
            table:nationality ?d.
        }""")

    winners = []
    for row in qres:
        if nation in ("%s %s" % row):
            winners.append(("%s %s" % row).split("http",1)[0])

    return render_template("winners.html", winners = winners)

@app.route("/nobel/getCategory", methods=['GET', 'POST'])
def getCategory():
#GET: Return list of all nobel winners for the given nation
    category = request.form.get('category')
    qres = g.query(
        """
        PREFIX table:<http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>
        PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
        SELECT ?n ?w
        {
            ?g rdf:type table:PersonWinner;
            table:name ?n;
            table:WonPrize ?w.
        }
        GROUP BY ?w
        ORDER BY ?w""")


    winners = []
    for row in qres:
        if category in ("%s %s" % row):
            winners.append(("%s %s" % row).split("http", 1)[0])

    return render_template("winners.html", winners = winners)

@app.route("/nobel/getYearCategory", methods=['GET', 'POST'])
def getYearCategory():
#GET: Return list of all nobel winners for the given year and category
    category = request.form.get('category')
    year = str(request.form.get('year'))
    qres = g.query(
        """
        PREFIX table:<http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>
        PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
        SELECT ?n ?w ?y
        {
            ?g rdf:type table:PersonWinner;
            table:name ?n;
            table:WonPrize ?w.
            ?w table:yearWon ?y.
        }
        GROUP BY ?w
        ORDER BY ?w""")

    winners = []
    for row in qres:
        if (category in ("%s %s %s" % row)) and (year in ("%s %s %s" % row)):
            winners.append(("%s %s %s" % row).split("http", 1)[0])


    return render_template("winners.html", winners=winners)

@app.route("/getDetails/<string:name>")
def getDetails(name):
#GET: Return details of the given nobel winner
    name = name
    qres = g.query(
    """
    PREFIX table:<http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>
    PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
    SELECT ?n ?nt ?b ?p ?w ?y
    {
        ?g rdf:type table:PersonWinner;
        table:name ?n;
        table:nationality ?nt;
        table:birthYear ?b;
        table:photo ?p;
        table:WonPrize ?w.
        ?w table:yearWon ?y;
        
    }
    """)
    result = {
        "name": name,
        "nationality": 'not found',
        "category": 'not found',
        "year": 0000,
        "association": 'not found',
        "born": 0000,
        "died": 0000,
        "motivation": 'not found',
        "photo": 'not found'
    }
    for row in qres:
        if name in str(row.asdict()['n'].toPython()):
            #result["association"] = str(row.asdict()['a'].toPython()).split('#',1)[-1]
            result["nationality"] = str(row.asdict()['nt'].toPython()).split('/')[-1]
            result["born"] = str(row.asdict()['b'].toPython())
            #result["died"] = str(row.asdict()['d'].toPython())
            result["photo"] = str(row.asdict()['p'].toPython())
            #result["motivation"] = str(row.asdict()['m'].toPython())
            result["category"] = str(row.asdict()['w'].toPython()).split('/')[-4]
            result["year"] = str(row.asdict()['y'].toPython())


    qres2 = g.query(
        """
        PREFIX table:<http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>
        PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
        SELECT ?n ?a
            {
                ?g rdf:type table:PersonWinner;
                table:name ?n;
                table:Association ?a.
            }

        """)

    for row in qres2:
        if name in str(row.asdict()['n'].toPython()):
            result["association"] = str(row.asdict()['a'].toPython()).split('#',1)[-1]


    qres3 = g.query(
    """
    PREFIX table:<http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>
    PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
    SELECT ?n ?d
        {
            ?g rdf:type table:PersonWinner;
            table:name ?n;
            table:deathYear ?d.
        }

    """)


    for row in qres3:
        if name in str(row.asdict()['n'].toPython()):
            result["died"] = str(row.asdict()['d'].toPython())


    qres4 = g.query(
        """
        PREFIX table:<http://swat.cse.lehigh.edu/resources/onto/nobel.owl#>
        PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX xsd:<http://www.w3.org/2001/XMLSchema#>
        SELECT ?n ?m
        {
            ?g rdf:type table:PersonWinner;
            table:name ?n;
            table:WonPrize ?w.
        ?w table:motivation ?m;
        }

        """)

    for row in qres4:
        if name in str(row.asdict()['n'].toPython()):
            result["motivation"] = str(row.asdict()['m'].toPython())

    return render_template("details.html", name=name, association=result["association"], nationality=result["nationality"], born=result["born"], died=result["died"], photo=result["photo"], motivation=result["motivation"], category=result["category"], year=result["year"] )

if __name__ == "__main__":
    app.run()

graph has 7780 statements.
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
On macOS, try disabling the 'AirPlay Receiver' service from System Preferences -> Sharing.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/werkzeug/serving.py", line 911, in prepare_socket
    s.bind(server_address)
OSError: [Errno 48] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-a319b7287cb9>", line 300, in <module>
    app.run()
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1188, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "/opt/anaconda3/lib/python3.8/site-packages/werkzeug/serving.py", line 1062, in run_simple
    s = prepare_socket(hostname, port)
  File "/opt/anaconda3/lib/python3.8/site-packages/werkzeug/serving.py", line 930, in prepare_socket
    sys.exit(1)
SystemExit: 1

During handling of the above exception,

TypeError: object of type 'NoneType' has no len()